# Forecasting the Fonterra Farmgate Milk Price

Oscar Dowson*, Department of Engineering Science, University of Auckland

*odow003@aucklanduni.ac.nz

## Introduction

New Zealand is the world's largest exporter of dairy commodity products, representing approximately one third of international dairy trade. The contemporary New Zealand milk processing sector is dominated Fonterra, a large processing co-operative who collect 85% of the milk produced in New Zealand. Fonterra collect milk from their 10,500 farmer shareholders and process it into a range of commodity products (such as whole milk powder), as well as value-added products such as specialty cheeses, yoghurts and icecream.

Suppliers on a basis of kilogram of milk solids (milkfat plus protein) supplied. However, due to the de-facto monopsony of Fonterra in New Zealand, and the small domestic market relative to production (95% is exported), there is no "fair market price" for milk produced in New Zealand. Therefore, the New Zealand Commerce Comission regulates the process by which Fonterra calculates the price it pays its farmer shareholders. This is broken into two components: the Farmgate Milk Price (FMP) and the dividend.

The FMP corresponds to the international price that an efficient producer of dairy commodity products could achieve on the international market, while the dividend corresponds to the return on capital that the farmers have invested in Fonterra in order to produce added value products.

The FMP and dividend are determined \emph{ex-post} at the end of each season (which begins on June 1 each year). Therefore, during the season, farmers are uncertain about the price they will receive for milk they will produce during the season. Fonterra publish a forecast for the FMP at least every quarter, but historically these forecasts had large uncertainties of up to $\pm50\%$.

The goal of this paper is to create a probabilistic model for the FMP. The FMP was chosen over the combined Farmgate Milk Price plus Dividend, because the FMP is the major component of the total price farmers receive (as a rule of thumb, the split between the FMP and dividend is on the order of of a 10 to 1). It is also the easier part to model since it is largely driven by (mostly) publically available information and a regulated process. In contrast, the dividend is derived from confidential internal sales figures and can be set by the board of Fonterra based on political and business decisions.

**Global Dairy Trade**

Since the very beginning, the international dairy commodity market has been dominated by regulation, protectionism and opaque price discovery. A key reason for this was the perishable nature of milk products, and strong domestic consumption in the milk producing regions of the world. In short, because most milk was drunk at home in the form of liquid milk or processed products like cheese and yoghurt, there was little incentive to develop well-functioning international commodity markets.

However, as producers with liberal economic ideologies like New Zealand grew to dominate the international export market (representing a third of international exports despite producing only 3% of the total volume and exporting 95% of their domestic production), they began to want a new mechanism to discover the ''true'' international price of milk products.

For these reasons, Fonterra established Global Dairy Trade (GDT) as an independent subsidiary in 2008. GDT is an international auction based marketplace for dairy commodity products. The platform positioned itself as an independent, trusted and transparent trading platform and has in many ways exceeded all expectations. It has become the de-facto reference price for dairy related commodities.

The GDT auction is as an ascending price clock auction. For those readers who are not familiar with an ascending price clock auction, it begins with the sellers submitting an offer quantity (supply) and a price is set at which it is expected there will be more demand than the available the offer quantity. After being notified of the initial price, buyers submit demand quantities for each product. If this quantity exceeds the total available supply then, in a series of rounds, the price is slowly incremented, and buyers are given the opportunity to reduce the quantity they wish to purchase.  The auction ends when the total demand of the buyers is equal to the sellers’ offer quantity (set at the open of the auction). All buyers pay the final price.

Since milk is a perishable product, sellers are guaranteed to find a buyer for almost (depending on the size of the price increment) all the milk they wish to sell. Furthermore, since buyers can only decrease their purchase quantity, they must enter the auction demanding close to what they want to purchase.

There are 24 auctions each year. These are held on the first and third Tuesdays of each month.

In [ ]:
# Cells like this contain code that is used to analyse the data.
# You need to install Julia v0.5 and the IJulia.jl package if you want to execute them yourself
# (You'll also need the data...)

# Lets load some helpful packages
#  For dealing with tabular data
using DataFrames, DataFramesMeta
#  Interacting with the web
using JSON, Requests
#  Plotting
using Plots, StatPlots, PlotRecipes, RecipesBase
#  and doing some proper statistics
using Distributions, GLM, StatsBase

# We're also going to create some helper functions to simplify the path structure
function getdata(s, datekey=nothing, datestr="yyyy-mm-dd")
    df = readtable(joinpath(dirname(pwd()), "data", s))
    if !(datekey == nothing)
        df[datekey] = convert(DataArray{Date, 1}, map(d->Dates.Date(d, datestr), df[datekey]))
    end
    df
end
modelpath(s) = joinpath(dirname(pwd()), "data/model_parameters", s)
datapath(s) = joinpath(dirname(pwd()), "data", s)
docpath(s) = joinpath(dirname(pwd()), "docs", s)

# And we're going to use this function to de-serialise a DataFrame from JSON.
function load_df_from_json(d::Dict)
    df = DataFrame(hcat(d["columns"]...))
    names!(df, map(Symbol, d["colindex"]["names"]))
    df
end

In [ ]:
function stripsuffix(s)
    for suf in ["st", "nd", "rd", "th"]
        s = replace(s, suf, "")
    end
    s
end
function grablatestagrihq()
    SEASON = 2017
    nzxdata = JSON.parse(String(get("https://dairy-tools.nzx.com/fgmp/calculator_widget.json?client_key=15491abd-1307-4caa-abaa-843bc827e8eb").data))
    fgmp = 0.0
    effective_date = Dates.today()
    for calc in nzxdata["calculations"]
        if calc["calculation_type"] == "forecast" || calc["calculation_type"] == "estimate"
            fgmp += calc["fgmpe"] * calc["rwmp"]
            effective_date = Dates.Date(stripsuffix(calc["effective_at"]), "U d, y")
        end
    end
    aghq = getdata("forecasts/agrihq.csv", :Date)
    append!(aghq, DataFrame(Date = effective_date, Season=SEASON, Forecast= round(fgmp, 2)))
    unique!(aghq)
    writetable(datapath("forecasts/agrihq.csv"), aghq)
end
try
    grablatestagrihq()
catch
    warn("Unable to get latest agrihq data")
end 

In [ ]:
url2string(url) = String(get(url).data)
gettables(page) = matchall(r"<table.*?>(.+?)</table>"s, page)
getrows(table) = matchall(r"<tr.*?>(.+?)</tr>"s, table)
getcells(row) = matchall(r"<td.*?>(.+?)</td>"s, row)
getcellcontent(cell) = strip(match(r"<td.*?>(.+?)</td>"s, cell)[1])

In [ ]:
function extractforecasts(table, product)
    rows = getrows(table)
    trading_periods = [getcellcontent(cell) for cell in getcells(rows[2])]
    trading_periods[5] = "12 Month"
    Event   = String[]
    CP      = Int[]
    Quantity = String[]
    for (cp, row) in enumerate(rows[5:10])
        for (i, cell) in enumerate(getcells(row)[1:5])
            push!(Event, trading_periods[i])
            push!(CP, cp)
            push!(Quantity, getcellcontent(cell))
        end
    end
    return DataFrame(
        Date=fill(Dates.today(), length(CP)),
        Event = Event,
        CP = CP,
        Product = fill(product, length(CP)),
        Quantity= Quantity,
        Type = fill("Forecast", length(CP))
    ), trading_periods
end

function extractactuals(table, product, event)
    rows = getrows(table)
    DataFrame(
        Date=fill(Dates.today(), 6),
        Event = fill(event, 6),
        CP = 1:6,
        Product = fill(product, 6),
        Quantity = [String(getcellcontent(cell)) for cell in getcells(rows[4])],
        Type = fill("Actual", 6)
    )
end
function getactualofferquantity(product)
    page = url2string("http://www.nzxfutures.com/dairy/events/GDT$(product)/details")
    tables = gettables(page)
    tableindex = (length(tables) == (product=="SMP"?4:3))?2:1
    (forecasts, trade_events) = extractforecasts(tables[tableindex], product)
    if tableindex == 2        
        event = "TE $(parse(Int, trade_events[1][(end-2):end]) - 1)"
        append!(forecasts, extractactuals(tables[1], product, event))
    end
    forecasts
end
function updatequantityforecats()
    df = getactualofferquantity("WMP")
    for product in ["BMP", "BUT", "SMP", "WMP"]
        append!(df, getactualofferquantity(product))
    end
    df[:Quantity] = convert(DataArray{Int}, map(d->parse(Int, replace(d, ",", "")), df[:Quantity]))
    df[:CP] = convert(DataArray{String}, map(s->"$s", df[:CP]))
    stored_df = getdata("fonterra/forecasts.csv", :Date)

    for row in 1:size(df, 1)
        tmp_row = @where(stored_df, :Event .== df[row, 2], :CP .== df[row, 3], :Product .== df[row, 4], :Type .== df[row, 6])
#         if size(tmp_row, 1) == 0
            # new forecast
            append!(stored_df, df[row, :])
#         elseif tmp_row[end, :Quantity] != df[row, :Quantity]
            # updated forecast
            append!(stored_df, df[row, :])
#         end
    end
    unique!(stored_df)
    sort!(stored_df, cols=[:Date, :Product, :Event, :CP])
    writetable(datapath("fonterra/forecasts.csv"), stored_df)
    writetable(docpath("data/fonterra_quantity_forecasts.csv"), stored_df)
end
try
    updatequantityforecats()
catch
    warn("Failed to update quantity forecasts") 
end

In [ ]:
function getnthtuesday(year, month, n)
    for day in 1:7
        d = Dates.Date(year, month, day + (n-1)*7) 
        if Dates.dayofweek(d) == 2
            return d
        end
    end
    return Dates.Date(year, month, 1)
end
function getdateofauction(n)
    te183 = Dates.Date(2017,3,1)
    nth = isodd(n)?1:3
    months = floor(Int, (n - 183)/2)
    te_nth = te183 + Dates.Month(months)
    getnthtuesday(Dates.year(te_nth), Dates.month(te_nth), nth)
end

function gettotalforecasts()
    forecasts = getdata("fonterra/forecasts.csv", :Date)
    forecasts[:Event] = map(x->begin
        if x[1:2] != "TE" && x != "12 Month"
            return "TE $x"
        else
            return x
        end 
    end, forecasts[:Event])

    sort!(forecasts, cols=[:Product, :Event, :Date])
    total_forecasts = @select(by(@where(forecasts, :CP .!= "Total"), [:Product, :Event, :Type]) do df
        _df_ = by(df, :CP) do _
            DataFrame(
            Quantity=_[:Quantity][end],
            Date=_[:Date][end]
            )
        end
        DataFrame(
            Quantity=sum(_df_[:Quantity]),
            Date=maximum(_df_[:Date]),
            CP = "Total"
        )
        end, :Date, :Event, :CP, :Product, :Quantity, :Type)
    append!(total_forecasts, forecasts)
    unique!(total_forecasts)
    sort!(total_forecasts, cols=[:Date, :Product, :Event])
    total_forecasts = @where(total_forecasts, :Event .!= "12 Month", :CP .== "Total")
    total_forecasts[:TE] = convert(DataArray{Int}, map(x->parse(Int, x[4:end]), total_forecasts[:Event]))
    total_forecasts[:AuctionDate] = map(getdateofauction, total_forecasts[:TE])
    total_forecasts
end
function getdfforecast(total_forecasts)
    dfforecast = join(@where(total_forecasts, :Type .== "Actual"), @where(total_forecasts, :Type .=="Forecast"), on=[:Product, :CP, :Event, :TE, :AuctionDate])
    rename!(dfforecast, :Date, :Actual_Date)
    rename!(dfforecast, :Date_1, :Forecast_Date)
    rename!(dfforecast, :Quantity, :Actual_Quantity)
    rename!(dfforecast, :Quantity_1, :Forecast_Quantity)
    delete!(dfforecast, :Type_1)
    dfforecast[:TEPrior] = map((a,b)->floor(Int, round(Int, (a-b).value / 7)/ 2+0.1), dfforecast[:AuctionDate], dfforecast[:Forecast_Date])
    # dfforecast[:RelativeError] = (dfforecast[:Actual_Quantity] ./ dfforecast[:Forecast_Quantity] - 1)*100
    dfforecast[:ActualError] = dfforecast[:Actual_Quantity] - dfforecast[:Forecast_Quantity]
    sort!(dfforecast, cols=[:Product, :Forecast_Date, :Event, :CP])
    dfforecast
end
function saveforecastquantities()
    total_forecasts = gettotalforecasts()
    dfforecast = getdfforecast(total_forecasts)
    data = Dict()
    for prod in ["WMP", "SMP"]
        prod_df = @where(dfforecast, :Product .== prod)
        data[prod] = Dict()
        tmp_df = @select(@where(prod_df, :Type .== "Actual"), :TE, :Actual_Date, :Actual_Quantity)
        unique!(tmp_df)
        data[prod]["actual"] = Dict("date" => tmp_df[:Actual_Date], "quantity" => tmp_df[:Actual_Quantity])

        for te in 1:4
            tmp_df = @where(prod_df, :TEPrior.==te)
            data[prod]["te$(te)"] = Dict(
                "date" => tmp_df[:Actual_Date],
                "error" => map(x->round(x, 2), tmp_df[:ActualError]),
                "quantity" => tmp_df[:Forecast_Quantity]
            )
        end
    end
    open(docpath("json/forecast_quantity.json"), "w") do file
        print(file, "forecast_quantity = ")
        JSON.print(file, data)
        println(file, ";")
    end
end
try
    saveforecastquantities()
catch
    warn("Error saving json/forecast_quantity.json")
end

function getcombineddf()
    gdt = getdata("gdt/events.csv", :Date)
    gdtprice = melt(gdt, [:Date, :Season, :TE])
    rename!(gdtprice, Dict(:Date=>:AuctionDate, :variable=>:Product, :value=>:Price))
    gdtprice[:Product] = map(s->"$s", gdtprice[:Product])

    total_forecasts = gettotalforecasts()
    dfforecast = getdfforecast(total_forecasts)

    combined_df = join(gdtprice, @where(dfforecast, :CP.=="Total"), on=[:AuctionDate, :Product, :TE])
    combined_df
end
function rungdtactualprices()
    combined_df = getcombineddf()
#     maxdate = maximum(@where(combined_df, :Type .== "Actual")[:AuctionDate])
    data = Dict()
    for prod in ["WMP", "SMP"]
        tmp_df = @select(@where(combined_df, :Product .== prod, :Type .== "Actual"), :AuctionDate, :Actual_Quantity, :Price)
        unique!(tmp_df)
        data[prod] = Dict("date" => tmp_df[:AuctionDate], "price" => tmp_df[:Price])
    end
    open(docpath("json/actual_gdt_prices.json"), "w") do file
        print(file, "actual_gdt_prices = ")
        JSON.print(file, data)
        println(file, ";")
    end
end

try
    rungdtactualprices()
catch
    warn("Error saving json/actual_gdt_prices.json")
end

In [ ]:
total_forecasts = gettotalforecasts()
dfforecast = getdfforecast(total_forecasts)
@where(dfforecast, :Event .== "TE 185", :Product .== "SMP")

In [ ]:
function savefonterraforecastquantities()
    combined_df = getcombineddf()
    total_forecasts = gettotalforecasts()
    historic_accuracies = by(@where(combined_df, :Actual_Date .>=Dates.Date(2013,1,1)), [:Product, :TEPrior]) do df
        DataFrame(
        lower = quantile(df[:ActualError], 0.1),
        middle = quantile(df[:ActualError], 0.5),
        upper = quantile(df[:ActualError], 0.9)
        )
    end
    data = Dict()
    for prod in ["WMP", "SMP"]
        latestdate = maximum(total_forecasts[:Date])
        tmp_df = @where(total_forecasts, :AuctionDate .>= Dates.today(), :Type .== "Forecast", :Product .== prod)
        tmp_df = by(tmp_df, :Event) do _
            _[end, :]
        end
        @show tmp_df
        tmp_df = tmp_df[(end-3):end, :]

        tmp_df[:TEPrior] = 1:4
        tmp_df = join(tmp_df, historic_accuracies, on=[:Product, :TEPrior])
        data[prod] = Dict(
        "date"   => tmp_df[:AuctionDate],
        "lower"  => map(x->round(Int, x), tmp_df[:Quantity] + tmp_df[:lower]),
        "middle" => map(x->round(Int, x), tmp_df[:Quantity] + tmp_df[:middle]),
        "upper"  => map(x->round(Int, x), tmp_df[:Quantity] + tmp_df[:upper])
        )
    end
    open(docpath("json/fonterra_forecast_quantity.json"), "w") do file
        print(file, "fonterra_forecast_quantity = ")
        JSON.print(file, data)
        println(file, ";")
    end
end
try
    savefonterraforecastquantities()
catch
    warn("Error saving json/fonterra_forecast_quantity.json")
end

In [ ]:
function saveseasonalisedquantitypredictions()
    combined_df = getcombineddf()
    combined_df[:Month] = map(d->Dates.Month(d).value, combined_df[:Actual_Date])
    tmp_df = @select(combined_df, :Product, :Month, :Actual_Quantity)
    unique!(tmp_df)
    sort!(tmp_df, cols=[:Product, :Month])
    average_quantity = by(tmp_df, [:Product, :Month]) do df
        DataFrame(
            lower = quantile(df[:Actual_Quantity], 0.1),
            middle = quantile(df[:Actual_Quantity], 0.5),
            upper = quantile(df[:Actual_Quantity], 0.9)
        )
    end
    data = Dict()
    for prod in ["WMP", "SMP"] 
        data[prod] = Dict()

        dates  = []
        lower  = []
        middle = []
        upper  = []
        for futurete in maximum(@where(combined_df, :Type .== "Actual")[:TE]) + (5:24)
            dte = getdateofauction(futurete) 
            push!(dates, dte)
            tb = @where(average_quantity, :Product .== prod, :Month .== Dates.month(dte))
            push!(lower, round(Int, tb[:lower][1]))
            push!(middle, round(Int, tb[:middle][1]))
            push!(upper, round(Int, tb[:upper][1]))
        end
        data[prod]["date"] = dates
        data[prod]["lower"] = lower
        data[prod]["middle"] = middle
        data[prod]["upper"] = upper
    end
    open(docpath("json/seasonalised_quantity.json"), "w") do file
        print(file, "seasonalised_quantity = ")
        JSON.print(file, data)
        println(file, ";")
    end
end
try
    saveseasonalisedquantitypredictions()
catch
    warn("Error saving json/seasonalised_quantity.json")
end

In [ ]:
# Since not all plotting backends support ysticks, we're going to create our own using a recipe!
@recipe function f(::Type{Val{:ysticks}}, x, y, z)
    xnew, ynew = zeros(3 * length(x)), zeros(3 * length(x))
    xnew[1:3:end] = x
    xnew[2:3:end] = x
    xnew[3:3:end] = NaN
    
    ynew[2:3:end] = y
    ynew[3:3:end] = NaN
    
    seriestype  := :path
    x           := xnew
    y           := ynew
    
    @series begin
        seriestype := :scatter
        x := x
        y := y
        label := ""
        primary := false
        markershape := d[:markershape]
        markerstrokewidth := d[:markerstrokewidth]
        ()
    end
    markershape := :none
    ()
end
Plots.@deps ysticks path scatter

In [ ]:
# The same thing goes for a correlogram!
@userplot correlogram
@recipe function f(cor::correlogram; partial=true, zvalue=1.96)
    x, lag = cor.args
    x = reshape(x, (length(x), 1))
    y = zeros(lag, 1)
    conf_int = zeros(lag)

    if partial
        StatsBase.pacf!(y, x, collect(1:lag))
        conf_int += zvalue / sqrt(length(x))
    else
        StatsBase.autocor!(y, x, collect(1:lag))
        for i=1:lag
            r = 0.
            for j=1:i
                r += y[j]^2
            end
            conf_int[i] = zvalue * sqrt(1 / length(x) * (1 + 2r))
        end
    end

    xguide --> "Lag"
    yguide --> (partial?"Partial Autocorrelation Coefficient":"Autocorrelation Coefficient")
    ylims  --> (-1, 1)
    @series begin
        seriestype  := :ysticks
        markershape := :circle
        markerstrokewidth := 0
        x           := 1:lag
        y           := y
        label := ""
        ()
    end
    @series begin
        seriestype  := :line
        x           := 1:lag
        y           := conf_int
        linestyle   := :dot
        linecolor       := colorant"darkgrey"
        label := ""
        ()
    end
    @series begin
        seriestype  := :line
        x           := 1:lag
        y           := -conf_int
        linestyle   := :dot
        label := ""
        linecolor       := colorant"darkgrey"
        ()
    end
end
Plots.@deps correlogram ysticks line

In [ ]:
# And a Q-Q plot!
@userplot qqplot
@recipe function f(qq::qqplot)
    dist, xx = qq.args
    Q = qqbuild(dist, xx)
    xguide --> "Theoretical Quantile"
    yguide --> "Sample Quantile"
    @series begin
        seriestype := :scatter
        x := Q.qx
        y := Q.qy
        label := ""
        markersize --> 2
        markerstrokewidth --> 0
        ()
    end
    
    Z = [min(minimum(Q.qx), minimum(Q.qy)), max(maximum(Q.qx), maximum(Q.qy))]
    @series begin
        seriestype := :line
        x := Z
        y := Z
        label := ""
        ()
    end
end
Plots.@deps qqplot scatter line


In [ ]:
# Now lets load our plotting backend and set a nice theme to use
pyplot()
theme(:sand)

## Fonterra Forecasts

Fonterra are in the best position to forecast the Farmgate Milk Price. They are privy to commercially sensitive data such as the exchange rate, product mix and sales curve that we have had to approximate. They do release a forecast at least every quarter (as mandated by DIRA 2001). In the graph below, we plot every forecast Fonterra has made over time (the x-axis). Each coloured line is a different season. The rightmost point on each line is the date and price at which the the Farmgate Milk Price was finalised.

In [ ]:
# First up, lets extract all the historic Fonterra forecasts
# We're going to encapsulate all the analysis in functions to minimise the quantity of objects in the global scope.
function get_fonterra_forecast()
    # load the dataframe
    fonterra_forecast = getdata("forecasts/fonterra.csv", :Date, "yyyy-m-d")
    # convert to Julia Dates
#     fonterra_forecast[:Date] = convert(DataArray{Date, 1}, map(d->Dates.Date(d, "yyyy-m-d"), fonterra_forecast[:Date]))
    # prettify the season string
    fonterra_forecast[:sSeason] = map(s->"$(s)/$(s - 2000 + 1)", fonterra_forecast[:Season])
    # calculate how many days since the start of the season (June 1st)
    fonterra_forecast[:Day] = map((d, s) -> (d - Dates.Date(s, 6, 1)).value, fonterra_forecast[:Date], fonterra_forecast[:Season])
    # sort based on date of the forecast
    sort!(fonterra_forecast, cols=[:Date])
    
    return fonterra_forecast
end

# grab the historic data
fonterra_forecast = get_fonterra_forecast()

# Plot it!
plot(fonterra_forecast,
    :Date,
    :Forecast,
    group=:sSeason,
    linetype=:step,
    xlabel="Date of Forecast",
    ylabel="Forecast FMP (\$/kgMS)"
)

Some seasons, such as the 2012/13 season, Fonterra's forecasts were close to the the final Farmgate Milk Price (i.e. within \$0.50). However, in other seasons, such as the 2009/10 or 2014/15 season, Fonterra has be out by more than \$2/kgMS in its initial forecast. In the graph below, we plot the error in Fonterra's forecasts against the number of days since the season began. We bound these errors by the maximum absolute error observed in the historical forecasts.

In [ ]:
# using the irregularly spaced historic forecasts, we're going to interpolate them using the most recent value
function get_daily_forecast(fonterra_forecast)
    # Calculate the final payout (last forecast recorded)
    final_payout = by(fonterra_forecast, :Season) do df
        DataFrame(FinalPayout = df[:Forecast][end])
    end
    # Drop this current season
    fonterra_forecast2 = join(@where(fonterra_forecast, :Season .< 2016), final_payout, on=[:Season])
    # Calculate the additive error
    fonterra_forecast2[:Error] = fonterra_forecast2[:Forecast] - fonterra_forecast2[:FinalPayout]
    
    # Calculate error for each season up to 500 days from 1 June
    by(fonterra_forecast2, :Season) do df
        # Initialise storage
        y = zeros(500)    
        # for each forecast
        for j=1:(size(df, 1)-1)
            day_start = max(1, df[:Day][j])
            day_stop  = max(1, df[:Day][j+1]-1)
            # interpolate
            y[day_start:day_stop] = df[:Error][j]    
        end
        # store the last
        y[df[:Day][end]:end] = df[:Error][end]
        # return a DataFrame
        DataFrame(day=1:500, Error=y)
    end
end

# Get the daily forecast
daily_forecast = get_daily_forecast(fonterra_forecast)

# Calculate the worst historic forecast at any given day
function get_error_bars(daily_forecast)
    by(daily_forecast, :day) do df
        DataFrame(
            PlusError=maximum(abs(df[:Error])),
            NegError=-maximum(abs(df[:Error]))
        )
    end
end

# Get the worst forecast
error_bars = get_error_bars(daily_forecast)

# Plot it!
#  the historic forecasts...
plot(daily_forecast, :day, :Error, group=:Season)
#  the positive error
plot!(error_bars, :day, :PlusError, label="Maximum Abs. Error", linewidth=2, c=colorant"darkblue")
#  the negative error
plot!(error_bars, :day, :NegError, label="", linewidth=2, c=colorant"darkblue")
#  fill in the plot labels
plot!(xlabel="Days since June 1st", ylabel="Error in Forecast (\$/kgMS)")

At the start of the season, Fonterra can be out by as much as \$2.50/kgMS. However, by the end of the season (i.e. in the following June), this error has narrowed to $\pm$\$0.2/kgMS. The reason for the residual error, even after the season has finished is the lag between production and sales, as well as a delay in finalising the average exchange rate achieved, and total costs. One may also conjecture that there is a political decision to delay the announcement of the final price until the annual results are announced in September, despite Fonterra internally being aware of a more accurate figure sometime between June and September.

We therefore have a model for the FMP by assuming the current Fonterra forecast, plus or minus the calculated maximum absolute error. We assume that this error is uniformly distributed. One reason that a uniform distribution was chosen was due to the lack of available data (we only have seven seasons since Fonterra changed the mechanism by which they set the FMP). There is no evidence that more complicated models of the uncertainty (such as the errors being normally, or triangularly distributed) provide more predictive power. For a similar reason, we chose to use the empirical error function, rather than fitting a smooth error function to the data.

In [ ]:
# extract the daily error
const fonterra_uniform_error = collect(error_bars[:PlusError])

# save it to JSON so we can use later
open(modelpath("fonterra_error.json"), "w") do file
    JSON.print(file, map(r->round(r, 3), fonterra_uniform_error))
end

# This function creates a Uniform distribution for possible prices based on Fonterra's current forecast
#    and the days_since_june_first
function create_fonterra_distribution(forecast, days_since_june_first)
    fonterra_uniform_error = JSON.parsefile(modelpath("fonterra_error.json"))
    err = fonterra_uniform_error[days_since_june_first]
    Uniform(forecast-err, forecast+err)
end

### Other Forecasters

In addition to Fonterra, there are other analysts in the market who forecast the Farmgate Milk Price. Chiefly, these are the major banks since their lending criteria depends a lot on forecasting the return a given farmer will make. We currently record the forecasts of five major analysts who produce public forecasts on a semi-regular basis:

1. [AgriHQ](https://agrihq.co.nz/toolbox/farmgate-milk-price-calculator/)
2. [ASB](http://reports.asb.co.nz/report-list/channel/4035/0/0/rural-economic-note.html)
3. [BNZ](https://www.bnz.co.nz/institutional-banking/research/publications#publication-Rural_Wrap)
4. [NZX](http://www.nzxfutures.com/dairy/quotes/mkp)
5. [Westpac](https://www.westpac.co.nz/agribusiness/agri-information/fortnightly-agri-updates/).

We assume that each of these forecasts have the same historical uncertainty as the Fonterra forecast. 


In [ ]:
function getanalystforecast(analyst, season, date)
    # look up the empirical error function
    fonterra_uniform_error = JSON.parsefile(modelpath("fonterra_error.json"))
    # load the analysts forecasts
    df = getdata("forecasts/$(analyst).csv", :Date)
    # convert the Date to Julian format
#     df[:Date] = convert(DataArray{Date}, map(i->Date(i, "yyyy-mm-dd"), df[:Date]))
    # get the most recent forecast
    forecast = @where(df, :Season.==season, :Date .<= date)[:Forecast][end]
    # calculate the additive error
    err = fonterra_uniform_error[(date - Dates.Date(season, 6,1)).value]
    # return probability distribution
    return Uniform(forecast-err, forecast+err)
end

## The Farmgate Milk Price as a weighted sum of GDT auctions

In actuality, Fonterra calculate the Farmgate Milk Price as the weighted sum of the GDT auctions, converted into New Zealand dollars at an average exchange rate ($r$), less the costs incurred in production ($c$). 

$$FMP = \frac{\sum\limits_{t=1}^{T}\sum\limits_{i\in I} w_t \lambda_{t,i} p_{t,i}}{r} - c,$$

where $I = \{AMF, BUT, BMP, SMP, WMP\}$, $w_t$ is the proportion of production sold in GDT auction $t$, $\lambda_{t,i}$ is the quantity of RCP $i$ produced in GDT auction period $t$ from 1kgMS, and $p_{t,i}$ is the average winning price for RCP $i$ in GDT auction $t$.

We then face five challenges

1. Estimating the sales curve $w_t$
2. Estimating the product mix $\lambda_{t,i}$
3. Forecasting the average exchange rate $r$
4. Forecasting the costs incurred by Fonterra $c$
5. Forecasting GDT Prices $p_{t,i}$

In the following sections we investigate each of these in turn.

### 1. Estimating the Sales Curve

The second important factor to consider is the timing of Fonterra's sales. In the Farmgate Milk Price Statement, Fonterra only provides sales data broken down by quarter. This proved to be too coarse for our analysis. However, Fonterra provide the cumulated contracted sales volume on a monthly basis in a graph format. This data was estimated from the graph and recorded as the approximate cumulative monthly contracts sales volume (in thousand metric tonnes) for the seasons 2011/12 through 2015/16. There is a small lag between supply, production and sales so that it is not until July of the following season (i.e. two months into the next season) that the final sales of the previous seasons production have been completed.

We can calculate the average sales curve my taking the mean of the cumulative proportion of total sales achieved in each month over the last five seasons and then calculating the difference between months.

In [ ]:
# Extract the monthly sales data
theme(:none)
function get_sales_data()
    df = getdata("fonterra/monthly_contracts.csv")
    # rename some columns
    names!(df, [:Month, Symbol("2011/12"), Symbol("2012/13"), Symbol("2013/14"), Symbol("2014/15"), Symbol("2015/16")])

    # go from cumulative to proportional
    for i=reverse(2:size(df, 1))
        for j in 2:size(df, 2)
            df[i, j] = df[i, j] - df[i-1, j]
        end
    end

    df = melt(df, :Month)
    df = join(df, by(df, :variable) do _df
        sum(_df[:value])
        end, on=[:variable])
    df[:Proportion] = df[:value] ./ df[:x1]
    df = @select(df, :Month, :Proportion, :variable)
    rename!(df, :variable, :Season)
    df
end
sales_data = get_sales_data()

unique_months = unique(sales_data[:Month])

function get_sales_curve(sales_data, unique_months)
    map(m->mean(@where(sales_data, :Month.==m)[:Proportion]), unique_months)
end

average_sales_curve = get_sales_curve(sales_data, unique_months)

plot(sales_data, :Month, :Proportion, group=:Season, xrotation=45,
xlabel="Month", ylabel="Proportion of Total Sales")
plot!(unique_months, average_sales_curve, label="Average Sales Curve", linewidth=4)
# savefig("salescurve.pdf")

The resulting sales curve is very noisy, particularly in December where very low sales quantities were coducted in 2014. The low volume of sales was likely driven by the low prices observed at the time. However, since milk is a perishable product, the production still had to be sold at some point in time which accounts the the large spike in January and February when the product that was held back was released to the market. We could improve the sales curve by smoothing out these bumps, but for simplicity we shall use the unmodified version.

In [ ]:
open(modelpath("sales_curve.json"), "w") do file
    JSON.print(file, map(r->round(r, 5), average_sales_curve))
end

function load_sales_curve()
    JSON.parsefile(modelpath("sales_curve.json"))
end

### 2. Estimating the product mix

As part of the annual Milk Price Statement, Fonterra release their assumed production of each RCP broken down by quarter. We can use this data to calculate the quantity of each RCP produced from 1kg of milk solids.

$$\lambda_i = \frac{Quantity\ RCP_i\ Produced}{Total\ Production},\quad i \in \{AMF, BUT, BMP, SMP, WMP\}$$

where $\lambda_i$ is the quantity of RCP $i$ produced from 1kg of milk solids.

This was done for all seasons and periods, and the average $\lambda_i$ was calculated for each RCP.

In [ ]:
function get_production_data()
    qprod = getdata("fonterra/quarterly_production.csv")

    for key in [:WMP, :SMP, :BUT, :AMF, :BMP]
        qprod[Symbol("$(key)p")] = qprod[key] ./ qprod[:Supply]
    end

    periods = ["Jul-Aug", "Sep-Nov", "Dec-Feb", "Mar-May"]
    qprod[:sPeriod] = ""
    @byrow! qprod :sPeriod = periods[:Period]
    qprod[:SeasonPeriod] = map((s,p)->string(s, " ",p), qprod[:Season], qprod[:sPeriod])
    qprod = @select(qprod, :Season, :sPeriod, :SeasonPeriod, :WMPp, :SMPp, :BUTp, :AMFp, :BMPp)
    names!(qprod, [:Season, :sPeriod, :SeasonPeriod, :WMP, :SMP, :BUT, :AMF, :BMP])

    melt(qprod, [:Season, :sPeriod, :SeasonPeriod])
end

production_data = get_production_data()

plot(production_data, :SeasonPeriod, :value, group=:variable,
xlabel="Time", ylabel="kg Produced per kgMS", xrotation=90)
# savefig("productmix.pdf")

There is clear evidence that the product mix follows a seasonal cycle where relatively more Whole Milk Powder is produced, and relatively less Skim Milk Powder is produced, during March to August than during September to February. These differences are driven by limitations in capacity (during the peak of production in October/November, Fonterra may be constrained by the perishable nature of milk and forced to convert it into less valuable products), or differences in value over time.

In [ ]:
function get_sales_mix(production_data)
    product_lambda = by(production_data, [:variable, :sPeriod]) do _df
            mean(_df[:value]) 
        end
    names!(product_lambda, [:product, :period, :weight])
    product_lambda
end
product_mix = get_sales_mix(production_data)

These values can be interpreted as follows: for every one kilogram of milk solids supplied to Fonterra during December and February, on average 1.15kg of WMP is produced, 0.42kg of SMP is produced and so on. These values sum to more than one because the input measurement (milk solids) does not include water, while the outputs (whole milk powder) does.

In [ ]:
open(modelpath("product_mix.json"), "w") do file
    JSON.print(file, product_mix)
end
function load_product_mix()
    pm = JSON.parsefile(modelpath("product_mix.json")) 
    df = load_df_from_json(pm)
    df[:product] = convert(DataArray{Symbol, 1}, df[:product])
    df
end

### 3. Estimating the Exchange Rate

Fonterra receives large quantities of Foreign currency receipts due to the fact GDT auctions are conducted in USD. However in order to pay the farmers, it is forced to convert these recepits back into New Zealand dollars. To reduce the risk surrounding the exchange rate conversion, Fonterra uses a range of forward foreign exchange contracts and currency options contracts. This hedging policy reduces the impact of the volatility in the New Zealand dollar on the FMP, but introduces an additional complexity for external modellers.

The only information that Fonterra releases about its achieved exchange rate (except in the Milk Price Statement at the end of each season) is one paragraph each year that reads as follows:

"As of 31 July [Current Year], Fonterra had foreign exchange contracts in place in respect to approximately [X] per cent of the USD equivalent operating cash flow exposure expected to impact on the Farmgate Milk Price for the [Next Season] Season. If the balance was hedged based on a spot exchange rate of [Y], the average USD:NZD conversion rate would be [Z] cents."

This contains enough information to calculate the average exchange rate already hedged by Fonterra, however due to the complex nature of the forward and options contracts, this average rate may vary as the spot price varies.

The average exchange rate achieved by Fonterra during a season is therefore: 

$$Average\ Exchange\ Rate = Z + (1-X)(\alpha -Y),$$

where $\alpha$ is the spot price at which Fonterra exchanges the remainder of its foreign currency receipts.

In [ ]:
function update_exchange_rate_data()
    page = reportdata = String(get("http://www.rbnz.govt.nz/statistics/b1").data)

    dates = match(r"<th class=\"title\" style=\"width\: 9.1\%\"></th>(.+?)</tr>"s, page)[1]
    rex = r"<em>(.+?)</em>"
    _keys = String[]
    for m in matchall(rex, dates)
        push!(_keys, match(rex, m)[1])
    end

    usd = match(r"<th class=\"title\".+?>United States dollar</td>(.+?)</tr>"s, page)[1]
    rex = r"<td.*?>(.+?)</td>"
    _values = Float64[]
    for m in matchall(rex, usd)
        push!(_values, parse(Float64, match(rex, m)[1]))
    end

    df = getdata("rbnz/exchange_rates.csv")
    for (_date, val) in zip(_keys, _values)
        append!(df, DataFrame(Date="$(Dates.Date(_date, "dd u yyyy"))", USD=val))
    end
    unique!(df)
    writetable(datapath("rbnz/exchange_rates.csv"), df)
end
update_exchange_rate_data()

In [ ]:
ex = getdata("rbnz/exchange_rates.csv", :Date)
# ex[:Date] = convert(DataArray{Date}, map(i->Date(i, "yyyy-mm-dd"), ex[:Date]))
ex[:Month] = map(Dates.month, ex[:Date])
ex[:Season] = 0
for i=1:size(ex, 1)
    if ex[i, :Month] >= 6
        ex[i, :Month] -= 5
        ex[i, :Season] = Dates.year(ex[i, :Date])
    else
        ex[i, :Month] += 7
        ex[i, :Season] = Dates.year(ex[i, :Date]) -1
    end
end

fonterra = DataFrame(
Season   = reverse([2015,      2014,      2013,      2012]),
actual   = reverse([0.7082,    0.7882,    0.8086,    0.7986]),
spot     = reverse([0.6798,    0.7329,    0.8430,    0.8200]),
fraction = reverse([0.67,      0.71,      0.67,      0.6]),
assumed  = reverse([0.6603,    0.8490,    0.7993,    0.8089]),
implied  = reverse([0.69,      0.826,     0.798,     0.78]),
Date = reverse([Date(2015,6,1), Date(2014,6,1), Date(2013,6,1), Date(2012,6,1)]),
)
f2 = copy(fonterra[end, :])
f2[:Date][end] += Dates.Year(1)
append!(fonterra, f2)
fonterra[:hedged] = (fonterra[:implied] - (1 - fonterra[:fraction]) .* fonterra[:assumed]) ./ fonterra[:fraction]

# w = DataFrame(Month=1:14, weight=average_sales_curve)
# w = DataFrame(Month=1:14, weight=ones(14)/14)

rbnz = deepcopy(ex)
rbnz2 = copy(@where(rbnz, :Month .<= 2))
rbnz2[:Month] += 12
rbnz2[:Season] -= 1
append!(rbnz, rbnz2)

# rbnz_mth = by(rbnz, [:Season, :Month]) do df
#     DataFrame(USD = mean(df[:USD]))
# end
# rbnz_mth = join(rbnz_mth, w, on=[:Month])

spot = by(rbnz, :Season) do df
#     DataFrame(CalculatedSpot=dot(df[:USD], df[:weight]))
#     DataFrame(CalculatedSpot=mean(df[:USD]))
    DataFrame(CalculatedSpot=quantile(df[:USD], 0.5))
end

estimates = join(fonterra, spot, on=[:Season])

estimates[:Estimated] = estimates[:fraction] .* estimates[:hedged] + (1-estimates[:fraction]) .* estimates[:CalculatedSpot]
estimates[:Error] = estimates[:Estimated] - estimates[:actual]
fonterra[:Estimated] = estimates[:Estimated]
fonterra[:Estimated][end] = fonterra[:Estimated][end-1]
estimates = @select(estimates, :Season, :Date, :actual, :Estimated, :Error, :CalculatedSpot)

plot(@where(ex, :Date.>=Date(2012,6,1), :Date.<=Date(2016,6,1)), :Date, :USD, label="Spot Rate")

plot!(fonterra, :Date, :spot, linetype=:step, linewidth=2, label="Fonterra: Sales Avg. Spot")
plot!(estimates, :Date, :CalculatedSpot, linetype=:step, linewidth=2, label="Calculated: Sales Avg. Spot")

plot!(fonterra, :Date, :actual, linetype=:step, linewidth=3, label="Fonterra: Achieved Conversion Rate")
plot!(fonterra, :Date, :Estimated, linetype=:step, linewidth=3, label="Calculated: Average Conversion Rate")
plot!(ylabel="USD:NZD Exchange Rate")

In the figure above, we have plotted the daily USD:NZD spot price over time, as well as the actual conversion rate achieved by Fonterra (reported _ex-post_) and the imputed average conversion rate. As a simplifying assumption, we assume that the average rate at which Fonterra converts the remainder of its unhedged foreign currency requirements is at the median spot price observed during the season.

We have glossed over a complication that the percentage reported by Fonterra is based on their _expected_ foreign exchange requirements. Large deviations (due to differences in production or sales prices) will impact the reliability of that assumption. One anecdoetal case for this is the 2015/16 season, in which Fonterra reported in September having 67% of their expected foreign currecy needs hedged as of July 31st. Given their forecast at the time of \$4.60 (compared to the final payout of \$3.90), it is likely that they overestimated the quantity of foreign currecy they would need to exchange. Increasing the percentage already hedged increases our estimate of the final average conversion rate, bringing it closer to that reported by Fonterra.

Estimating Fonterra's estimation of its future currency needs proved difficult so we leave that to future work. We also call on them to release additional information in a more timely fashion. In practice it is likely up to the practitioner to apply their own judgement as to whether it is likely that Fonterra misjudged the quantity of foreign currency it would need to exchange.

In [ ]:
immutable ExchangeRate{T} <: Distributions.Sampleable{Univariate,Continuous}
    proportion::Distributions.Uniform{T}
    spot::Distributions.Uniform{T}
    original_proportion::T
    hypothetical_spot::T
    implied_exchange_rate::T
end

In [ ]:
# An empirical estimator for the exchange rate
function Base.rand{T}(ex::ExchangeRate{T})
    proportion = rand(ex.proportion)
    spot = rand(ex.spot)
    alpha = (ex.implied_exchange_rate - (1-ex.original_proportion) * ex.hypothetical_spot) / ex.original_proportion
    alpha * proportion + (1-proportion) * spot
end

### 4. Estimating Fonterra's Cash and Capital Costs

The fourth challenge is estimating Fonterra's cash and capital costs. Although the smallest component of the three, lactose exhibits the largest variability. 

In [ ]:
function get_costs()
    summary = getdata("fonterra/summary.csv")
    summary[:Total] = summary[:Lactose_Cost] + summary[:Cash_Costs] + summary[:Capital_Costs]
    @select(summary, :Season, :Total, :Lactose_Cost,:Cash_Costs, :Capital_Costs)
end

fonterra_costs = get_costs()
    
plot(melt(fonterra_costs, :Season), :Season, :value, group=:variable)

We were unable to find any predictive correlates for the total costs. We leave the improvement of this model to future work.

A distribution for Fonterra's costs is the uniform distribution over $[1.93, 2.39]$.

In [ ]:
cost_distribution = Uniform(minimum(fonterra_costs[:Total]), maximum(fonterra_costs[:Total]))
open(modelpath("cost_distribution.json"), "w") do file
    JSON.print(file, cost_distribution)
end

# function load_cost_distribution()
#     d = JSON.parsefile(modelpath("cost_distribution.json")) 
# #     Uniform(d["a"], d["b"])
#     TriangularDist(d["a"], d["b"], 2.05)
# end
function load_cost_distribution(thedate)
    cost = getdata("forecasts/cost.csv", :Date)
    cost = @where(cost, :Season .== 2016, :Date .< thedate)
    Uniform(cost[end, :Minimum], cost[end, :Maximum])
end

### 5(a). Forecasting GDT Prices - using a random walk

First we obtained the average sales price (USD/Tonne) for each RCP in GDT auctions between July 2010 and July 2016. Since both Butter and Butter Milk Powder were either not traded, or irregularly traded during this time, a linear regression was used to impute the values for which there was no data.

In [ ]:
function get_imputed_gdt()
    gdt = getdata("gdt/events.csv", :Date)
#     gdt[:Date] = convert(DataArray{Date, 1}, map(d->Dates.Date(d, "yyyy-m-d"), gdt[:Date]))

    # Impute BMP
    rename!(gdt, :BMP, :BMPold)
    bmp_train = @where(gdt, :BMPold .!= "n.a.")
    bmp_train[:BMPold] = convert(DataArray{Float64}, map(s->parse(Float64, s), bmp_train[:BMPold]))
#     bmp_model = fit(LinearModel, @formula(BMPold~AMF+SMP+WMP), bmp_train)
    bmp_model = fit(LinearModel, BMPold~AMF+SMP+WMP, bmp_train)
    
    bmp_test = @where(gdt, :BMPold .== "n.a.")
    gdt[:BMP] = 0.
    gdt[:BMP][gdt[:BMPold] .== "n.a."] = predict(bmp_model, bmp_test)
    gdt[:BMP][gdt[:BMPold] .!= "n.a."] = map(i->parse(Float64, i), gdt[:BMPold][gdt[:BMPold] .!= "n.a."])

    # Impute BUT
    rename!(gdt, :BUT, :BUTold)
    but_train = @where(gdt, :BUTold .!= "n.a.")
    but_train[:BUTold] = convert(DataArray{Float64}, map(s->parse(Float64, s), but_train[:BUTold]))
#     but_model = fit(LinearModel, @formula(BUTold~AMF+SMP+WMP), but_train)
    but_model = fit(LinearModel, BUTold~AMF+SMP+WMP, but_train)
    but_test = @where(gdt, :BUTold .== "n.a.")
    gdt[:BUT] = 0.
    gdt[:BUT][gdt[:BUTold] .== "n.a."] = predict(but_model, but_test)
    gdt[:BUT][gdt[:BUTold] .!= "n.a."] = map(i->parse(Float64, i), gdt[:BUTold][gdt[:BUTold] .!= "n.a."])
    gdt
end

We can calculate the sales average price of milk in a GDT auction $y_t$ (USD/Tonne) as:

$$y_t = \sum\limits_{i\in I} \lambda_i(t) P^{(i)}_t,$$

where $I = \{AMF, BUT, BMP, SMP, WMP\}$, $\lambda_i(t)$ is the product mix coefficient for RCP $i$ in GDT auction $t$ (calculated earlier) and $P^{(i)}_t$ is the average sales price per tonne.


In [ ]:
function get_spot_price()
    gdt = get_imputed_gdt();
    # product_lambda
    product_mix = load_product_mix()
    names!(product_mix, [:variable, :Month, :weight])

    mth_groups = ["Dec-Feb","Dec-Feb","Mar-May",
    "Mar-May","Mar-May","Jul-Aug",
    "Jul-Aug","Jul-Aug","Sep-Nov",
        "Sep-Nov","Sep-Nov","Dec-Feb"]
    gdt[:Month] = map(m->mth_groups[Dates.month(m)], gdt[:Date])

    product_mix[:variable] = convert(DataArray{Symbol, 1}, product_mix[:variable])

    spot_price = by(
        join(
            melt(
                @select(gdt, :TE, :Date, :Season, :Month, :AMF, :BUT, :BMP, :SMP, :WMP),
                [:TE, :Date, :Season, :Month]
            ),
            product_mix,
            on=[:Month, :variable]),
        :Date) do df
            dot(df[:value], df[:weight]) 
    end
    names!(spot_price, [:Date, :SpotPrice])
    spot_price
end

spot_price = get_spot_price()

plot(spot_price, :Date, :SpotPrice, linetype=:step, legend=false,
xlabel="Date", ylabel="Product Mix Weighted GDT Price (USD/Tonne)")

In [ ]:
correlogram(log.(collect(spot_price[:SpotPrice])), 5, partial=true)

If we plot the partial auto-correlation coefficients, there is strong evidence of a log AR(2) effect.

$$log(y_t) = \alpha_0 + \alpha_1 log(y_{y-1}) + \alpha_2 log(y_{t-2}) + \varepsilon_t,\quad \varepsilon_t \sim \mathcal{N}(0, \sigma^2)$$

We can fit the $\alpha$ parameters using linear regression.

In [ ]:
logdata = DataFrame(
yt  = log.(spot_price[:SpotPrice][3:end]),
yt1 = log.(spot_price[:SpotPrice][2:(end-1)]),
yt2 = log.(spot_price[:SpotPrice][1:(end-2)])
)
# logmodel = fit(LinearModel, @formula(yt~yt1+yt2), logdata)
logmodel = fit(LinearModel, yt~yt1+yt2, logdata)

If we consider the qq-plot of the residuals (below), there is some evidence that the residuals are not normal. Indeed there is some evidence that the middle of the distribution has a smaller standard deviation that the standard deviation of all of the residuals, while the tails are fatter than could be expected from a normal distribution. One reason for this might be that the sequence of spot prices is typically characterised by small incremental changes, punctuated by large shocks to the system.

In [ ]:
qqplot(Normal(0, std(residuals(logmodel))), residuals(logmodel))

One solution to the poor fit is to sample from the empirical distribution of errors rather than from the fitted Normal distribution.

We now have a model for simulating future GDT prices.

In [ ]:
immutable GDTAR2 <: Distributions.Sampleable{Univariate,Continuous}
    coeffs::Vector{Float64}
    sales_curve::Vector{Float64}
    empirical_errors::Vector{Float64}
    observations_to_date::Vector{Float64} # includes the previous two from the last season
end

In [ ]:
immutable GDTAR2b <: Distributions.Sampleable{Univariate,Continuous}
    coeffs::Vector{Float64}
    sales_curve::Vector{Float64}
    empirical_errors::Vector{Float64}
    observations_to_date::Vector{Float64} # includes the previous two from the last season
end

In [ ]:
function nextspot(yt1, yt2, coeffs, errs)
    exp(coeffs[1] + coeffs[2] * log(yt1) + coeffs[3] * log(yt2) + rand(errs)) 
end
function simulate_prices(t::Int, yt1::Float64, yt2::Float64,
    coeffs::Vector{Float64}, w::Vector{Float64}, errs::Vector{Float64})
    z = 0.0
    for i=t:length(w)
        y = nextspot(yt1, yt2, coeffs, errs)
        z += y * w[i]
        yt2 = yt1
        yt1 = y
    end
    z
end
function Base.rand(g::GDTAR2)
    t = length(g.observations_to_date)
    (dot(g.observations_to_date, g.sales_curve[1:t]) +
    simulate_prices(t+1, g.observations_to_date[end], g.observations_to_date[end-1], g.coeffs, g.sales_curve, g.empirical_errors))/1000.
end
function Base.rand(g::GDTAR2b)
    t = length(g.observations_to_date) - 2
    (dot(g.observations_to_date[3:end], g.sales_curve[1:t]) +
    simulate_prices(t+1, g.observations_to_date[end], g.observations_to_date[end-1], g.coeffs, g.sales_curve, g.empirical_errors))/1000.
end

In [ ]:
open(modelpath("ar2_coefficients.json"), "w") do file
    JSON.print(file, coef(logmodel))
end
load_ar2_coefficients() = JSON.parsefile(modelpath("ar2_coefficients.json"))
open(modelpath("empirical_errors.json"), "w") do file
    JSON.print(file, residuals(logmodel))
end
load_empirical_errors() = JSON.parsefile(modelpath("empirical_errors.json"))

### 5(b). Forecasting GDT Prices - using the NZX Futures

A second source of information is available in order to forecast future GDT prices in NZX GDT Commodity Futures. These are financial futures contracts that settle against the average of the two GDT auctions in a month for AMF, BUT, SMP and WMP. Due to the small volumes, there is no contract for BMP.

In [ ]:
function gettype(x)
    if contains(x, "/NZXWMP")
        return "WMPfuture"
    elseif  contains(x, "/NZXSMP")
        return "SMPfuture"
    elseif  contains(x, "/NZXAMF")
        return "AMFfuture"
    elseif  contains(x, "/NZXBTR")
        return "BUTfuture"
    elseif  contains(x, "/NZXMKP")
        return "MKPfuture"
    elseif  contains(x, "/WMP")
        return "WMPoption"
    elseif  contains(x, "/MKP")
        return "MKPoption"
    end
end

function savereport(report)
    reportdata = String(gethttp("nzxfutures.com/system/dsp_reports/$report").data)
    filename = match(r".*/(.*?).csv", report)[1]
    open("$(filename).csv", "w") do f
        write(f, reportdata)
    end
    _df = readtable("$(filename).csv")
    rm("$(filename).csv")
    df = getdata("nzx/$(gettype(report)).csv")
    append!(df, _df)
    unique!(df)
    sort!(df, cols=[:Trade_date])
    writetable(datapath("nzx/$(gettype(report)).csv"), df)
end

gethttp(s) = get("http://$s")

function scrape_nzx()
    pagedata = String(gethttp("nzxfutures.com/dairy/market_info").data)
    for report in matchall(r"reports/([0-9]+?/original/\w+?-[0-9]+?-final\.csv)", pagedata)
        savereport(report)
    end

    df = getdata("nzx/MKPfuture.csv")
    mkp = @select(@where(df, :Code .== "MKPFU17"), :Trade_date, :Calculated_DSP)
    names!(mkp, [:Date, :Forecast])
    mkp[:Season] = 2016
    writetable(datapath("forecasts/mkp.csv"), mkp)
end

scrape_nzx()

In [ ]:
using Requests

function get_latest_futures()
    full_df = getdata("nzx/final_futures.csv")

    pagedata = String(get("http://nzxfutures.com/dairy/market_info").data)
    reports = matchall(r"downloads/([0-9]+?/NZX_(\w+)_Futures_FSP.csv)", pagedata)
    for report in reports
        filename = "tmp.csv"
        open(filename, "w") do file
            write(file, String(get("http://nzxfutures.com/system/$report").data))
        end
        df = readtable(filename)
        if isa(df[:Final_Settlement_Price], DataArray{String})
            df[:Final_Settlement_Price] = convert(DataArray{Float64}, map(p->parse(Float64, replace(p, ",", "")), df[:Final_Settlement_Price]))
        end
        r = df[:Settlement_Date] .== "23/08/2016"
        if sum(r) > 0 # incorrect data
            df[r, :Settlement_Date] = "23/09/2016"
        end
        rm(filename)
        append!(full_df, df)
        unique!(full_df)
    sort!(full_df, cols=[:Settlement_Date])
    writetable(datapath("nzx/final_futures.csv"), full_df)
    end
end
try
    get_latest_futures()
catch
    warn("Unable to get latest futures")
end

In [ ]:
function getfinalfutures()
    df = getdata("nzx/final_futures.csv")
    df[:Code] = map(d->length(d)==6?("F$(d[4:end])"):(d[4:end]), df[:Contract_Code])
    df[:Date] = convert(DataArray{Date}, map(d->Dates.Date(d, "dd/mm/yyyy"), df[:Settlement_Date]))
    df[:Prod] = map(d->d[1:3], df[:Contract_Code])
    df = @select(df, :Code, :Date, :Prod, :Final_Settlement_Price)
    df = unstack(df, :Prod, :Final_Settlement_Price)
    rename!(df, :BTR, :BUT)
    sort!(df, cols=[:Date])
    df = df[isna.(df[:BUT]) .< 0.5, :]
    df
end


In [ ]:
const codes = ["FF", "FG", "FH", "FJ", "FK", "FM", "FN", "FQ", "FU", "FV", "FX", "FZ"]
function getcode(s)
    for (idx, code) in enumerate(codes)
        if code == s[4:5]
            return idx
        end
    end
    return -1
end

function getfutures(key)
    df = getdata("nzx/$(key)future.csv")
    df[:Month] = 0
    df[:Month] = convert(Vector{Int}, map(getcode, df[:Code]))
    df[:Year] = 0
    @byrow! df :Year = 2000 + parse(Int, :Code[(end-1):end])
    df = @select(df, :Trade_date, :Month, :Year, :Calculated_DSP)
    rename!(df, :Calculated_DSP, key)
    df
end

function load_future_data()
    product_mix = load_product_mix()
    names!(product_mix, [:variable, :Month, :weight])
    product_mix[:variable] = convert(DataArray{Symbol, 1}, product_mix[:variable])

    product_lambda2 = join(product_mix, by(product_mix, :Month) do df
        DataFrame(Multiplier = sum(df[:weight]) / sum(@where(df, :variable .!= Symbol("BMP"))[:weight]))
        end, on=[:Month])
    product_lambda2[:weight] .*= product_lambda2[:Multiplier]
    product_lambda2 = @select(product_lambda2, :Month, :variable, :weight)

    historic_futures = getfinalfutures()#getdata("nzx/settled_futures.csv", :Date)
#     historic_futures[:Date] = convert(DataArray{Date, 1}, map(d->Dates.Date(d, "yyyy-m-d"), historic_futures[:Date]))
    mth_groups = ["Dec-Feb","Dec-Feb","Mar-May",
    "Mar-May","Mar-May","Jul-Aug",
    "Jul-Aug","Jul-Aug","Sep-Nov",
        "Sep-Nov","Sep-Nov","Dec-Feb"]
    historic_futures[:Month] = map(m->mth_groups[Dates.month(m)], historic_futures[:Date])

    historic_futures2 = join(melt(historic_futures, [:Code, :Date, :Month]), product_lambda2, on=[:variable, :Month])

    historic_futures2[:Spot] = historic_futures2[:value] .* historic_futures2[:weight]
    historic_spot = by(historic_futures2, :Date) do df
        DataFrame(Spot = sum(df[:Spot]))
    end
    historic_spot[:Month] = map(Dates.month, historic_spot[:Date])
    historic_spot[:Year] = map(Dates.year, historic_spot[:Date])
    historic_spot[:SeasonMonth] = 0
    historic_spot[:Season] = 0
    for i=1:size(historic_spot, 1)
        mth = historic_spot[i, :Month]
        yr = historic_spot[i, :Year]
        if mth < 6
            historic_spot[i, :SeasonMonth] = mth + 7
            historic_spot[i, :Season] = yr - 1
        else
            historic_spot[i, :SeasonMonth] = mth - 5
            historic_spot[i, :Season] = yr
        end
    end
    tmp = copy(@where(historic_spot, :SeasonMonth .<= 2))
    tmp[:SeasonMonth] += 12
    tmp[:Season] -= 1
    append!(historic_spot, tmp)
    sort!(historic_spot, cols=[:Season, :SeasonMonth])
    historic_spot = @select(historic_spot, :Season, :SeasonMonth, :Spot)

    futures = getfutures(:WMP)
    for key in [:SMP, :AMF, :BUT]
        futures = join(futures, getfutures(key), on=[:Trade_date, :Month, :Year])
    end
    futures[:Trade_date] = convert(DataArray{Date, 1}, map(d->Dates.Date(d, "yyyy-m-d"), futures[:Trade_date]))
    futures[:MonthIdx] = futures[:Month]
    futures[:Month] = map(m->mth_groups[m], futures[:MonthIdx])
    futures2 = join(melt(futures, [:Trade_date, :Month, :MonthIdx, :Year]), product_lambda2, on=[:Month, :variable])
    futures2[:Spot] = futures2[:value] .* futures2[:weight]
    futures2 = by(futures2, [:Trade_date, :MonthIdx, :Year]) do df
        DataFrame(Spot=sum(df[:Spot]))
    end
    futures2[:SeasonMonth] = 0
    futures2[:Season] = 0
    for i=1:size(futures2, 1)
        mth = futures2[i, :MonthIdx]
        yr = futures2[i, :Year]
        if mth < 6
            futures2[i, :SeasonMonth] = mth + 7
            futures2[i, :Season] = yr - 1
        else
            futures2[i, :SeasonMonth] = mth - 5
            futures2[i, :Season] = yr
        end
    end
    tmp = copy(@where(futures2, :SeasonMonth .<= 2))
    tmp[:SeasonMonth] += 12
    tmp[:Season] -= 1
    append!(futures2, tmp)
    sort!(futures2, cols=[:Season, :SeasonMonth])
    future_spot = @select(futures2, :Trade_date, :Season, :SeasonMonth, :Spot)
    historic_spot, future_spot
end

function getseasonmonth(date)
    mth = Dates.month(date)
    yr = Dates.year(date)
    if mth < 6
        mth += 7
        yr -= 1
    else
        mth -= 5
    end
    yr, mth
end

function get_historic_observations(historic_spot, date, season, minmonth)
    @where(historic_spot, :Season .== season, :SeasonMonth .< minmonth)
end

function get_future_observations(future_spot, date, season)
    #season, month = getseasonmonth(date)
    last_trade_date = maximum(@where(future_spot, :Trade_date .<= date, :Season .== season)[:Trade_date])
        @select(
            @where(future_spot, :Season.==season, :Trade_date.==last_trade_date),
            :Season, :SeasonMonth, :Spot
    )
end


function getnzxforecast(thedate)
    historic_spot, future_spot = load_future_data()

    nzx_data = get_future_observations(future_spot, thedate, 2016)
    
    append!(nzx_data, get_historic_observations(historic_spot, thedate, 2016, minimum(nzx_data[:SeasonMonth])))

    nzx_observations = join(nzx_data, DataFrame(SeasonMonth=1:14, weight=load_sales_curve()), on=[:SeasonMonth])
    dot(nzx_observations[:Spot], nzx_observations[:weight]) / 1000
end

thedate = Dates.today()
getnzxforecast(thedate)

In [ ]:
historic_spot, future_spot = load_future_data()

    nzx_data = get_future_observations(future_spot, thedate, 2016)
    
#     append!(nzx_data, get_historic_observations(historic_spot, thedate, minimum(nzx_data[:SeasonMonth])))

#     nzx_observations = join(nzx_data, DataFrame(SeasonMonth=1:14, weight=load_sales_curve()), on=[:SeasonMonth])
#     dot(nzx_observations[:Spot], nzx_observations[:weight]) / 1000

In [ ]:
futures = getfinalfutures()# getdata("nzx/settled_futures.csv", :Date)
# convertdate!(futures, :Date)
futures[:FirstDate] = map(d->Dates.Date(Dates.year(d), Dates.month(d), 1), futures[:Date])
futures = melt(futures, [:Code, :FirstDate, :Date])
futures[:Code] = map((v, c)->"$v$c", futures[:variable], futures[:Code])
futures = @select(futures, :Code, :Date, :variable, :value)
rename!(futures, :value, :Price)
rename!(futures, :variable, :Product)
futures[:Price] = convert(DataArray{Float64}, futures[:Price])

nzxproductmonths = ['F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X', 'Z']
function getfuturesforecast(product)
    df = getdata("nzx/$(product)future.csv", :Trade_date)
#     convertdate!(df, :Trade_date)
    df[:Month] = map(c->findfirst(nzxproductmonths, c[5]), df[:Code])
    df[:Year] = map(c->2000+parse(Int, c[6:7]), df[:Code])
    df[:Date] = map((y,m)->Dates.Date(y, m, 1), df[:Year], df[:Month])
    df[:Product] = convert(DataArray{Symbol}, map(d->Symbol(d[1:3]), df[:Code]))
    maxdate = maximum(df[:Trade_date])
    _df = @select(@where(df, :Trade_date .== maxdate), :Code, :Date, :Product, :Calculated_DSP)
    rename!(_df, :Calculated_DSP, :Price)
    _df
end
futureforecast = getfuturesforecast("WMP")
for product in ["SMP", "AMF", "BUT"]
    append!(futureforecast, getfuturesforecast(product))
end
futureforecast[:Code] = map(c->replace(c, "BTR", "BUT"),futureforecast[:Code])
futureforecast[:Product] = map(c->Symbol(replace("$c", "BTR", "BUT")),futureforecast[:Product])

data = Dict()
for product in unique(futures[:Product])
    data[product] = Dict()
    data[product]["historic"] = Dict()
    _ = @where(futures, :Product .== product)
    sort!(_, cols=[:Date])
    data[product]["historic"]["x"] = collect(_[:Date])
    data[product]["historic"]["price"] = collect(_[:Price])
    
    data[product]["forecast"] = Dict()
    _ = @where(futureforecast, :Product .== product)
    sort!(_, cols=[:Date])
    data[product]["forecast"]["x"] = collect(_[:Date])
    data[product]["forecast"]["price"] = collect(_[:Price])
end
open(docpath("json/futures.json"), "w") do file
    print(file, "future_data = ")
    JSON.print(file, data)
    print(file, ";")
end

## An ensemble forecast

We can then combine the following probabilistic forecasts:

1. (5%)  AgriHQ
2. (5%)  ASB
3. (5%)  BNZ
4. (5%)  NZX MKP
5. (5%)  Westpac
6. (25%) Fonterra Forecast
7. (25%) Log-normal AR(2) of GDT spot
8. (25%) NZX Futures Forecast

into an ensemble forecast. Provided the errors between them are independent, the average of these forecasts should outperform any individual forecast.


In [ ]:
function getanalyst_quantiles(analyst, season, date, N, quants)
    dist = getanalystforecast(analyst, season, date)
    quantile(rand(dist, N), quants)
end

function calculate_forecasts(thedate, exchange_rate_distribution, N, quants, saveforecasts=true)
    # Fonterra Estimates
    fonterra_distribution = getanalystforecast("fonterra", 2016, thedate)
    fonterra_quantiles = quantile(rand(fonterra_distribution, N), quants)
    
    # Analyst estimates
    agrihq_quantiles  = getanalyst_quantiles("agrihq", 2016, thedate, N, quants)
    asb_quantiles     = getanalyst_quantiles("asb", 2016, thedate, N, quants)
    bnz_quantiles     = getanalyst_quantiles("bnz", 2016, thedate, N, quants)
    mkp_quantiles     = getanalyst_quantiles("mkp", 2016, thedate, N, quants)
    westpac_quantiles = getanalyst_quantiles("westpac", 2016, thedate, N, quants)
    
    # Cost distribution
    cost_distribution = load_cost_distribution(thedate)

    # Draw N exchange rate samples
    exchange_rate_samples = rand(exchange_rate_distribution, N)
    
    # Draw N cost samples
    cost_samples = rand(cost_distribution, N)

    # GDT AR(2) estimator
    spot_price = get_spot_price()
    gdt_ar2_estimator = GDTAR2b(
        load_ar2_coefficients(),
        [wi/2 for wi in load_sales_curve() for i=1:2],
        load_empirical_errors(),
        collect(@where(spot_price, :Date .>= Dates.Date(2016,5,1), :Date .<= thedate)[:SpotPrice])
    )

    # NZX forecast
    nzx_futures_forecast = getnzxforecast(thedate)
    @show nzx_futures_forecast

    # Estimate GDT season average by autoregressive random walk
    autoregressive_gdt_forecasts = rand(gdt_ar2_estimator, N)
    autoregressive_forecasts = autoregressive_gdt_forecasts ./ exchange_rate_samples - cost_samples
    autoregressive_quantiles = quantile(autoregressive_forecasts, quants)
    
    # Use same distribution but shifted to NZX futures mean estimate
    nzx_futures_gdt_forecasts = autoregressive_gdt_forecasts + nzx_futures_forecast - mean(autoregressive_gdt_forecasts)
    nzx_futures_forecasts = nzx_futures_gdt_forecasts ./ exchange_rate_samples - cost_samples
    nzx_quantiles = quantile(nzx_futures_forecasts, quants)
    
    if saveforecasts
        append_auction_forecast(datapath("model_parameters/agrihq_forecasts.csv"), thedate, 2016,agrihq_quantiles)
        append_auction_forecast(datapath("model_parameters/asb_forecasts.csv"), thedate, 2016,asb_quantiles)
        append_auction_forecast(datapath("model_parameters/bnz_forecasts.csv"), thedate, 2016,bnz_quantiles)
        append_auction_forecast(datapath("model_parameters/mkp_forecasts.csv"), thedate, 2016,mkp_quantiles)
        append_auction_forecast(datapath("model_parameters/westpac_forecasts.csv"), thedate, 2016,westpac_quantiles)
        append_auction_forecast(datapath("model_parameters/fonterra_forecasts.csv"), thedate, 2016,fonterra_quantiles)
        append_auction_forecast(datapath("model_parameters/nzx_forecasts.csv"), thedate, 2016,nzx_quantiles)
        append_auction_forecast(datapath("model_parameters/ar_forecasts.csv"), thedate, 2016,autoregressive_quantiles)
    end
    auction_plus_quantiles = 
        0.0 * agrihq_quantiles + 
        0.05 * asb_quantiles + 
        0.05 * bnz_quantiles + 
        0.0 * mkp_quantiles + 
        0.0 * westpac_quantiles + 
        0.5 * fonterra_quantiles +
        0.2 * nzx_quantiles + 
        0.2 * autoregressive_quantiles
    return auction_plus_quantiles
end

function create_json(filename, cols)
    df = readtable(datapath("model_parameters/$(filename).csv"))
    outdict = Dict{Any, Any}()
    for sym in names(df)[3:end]
        outdict[sym] = Dict{Symbol, Any}(
            :x => df[:Date],
            :y => df[sym]
        )
    end
    open(docpath("json/$(filename).json"), "w") do file
        print(file, "$(filename) = ")
        JSON.print(file, outdict)
        print(file, ";\n")
    end
end

function append_auction_forecast(filename, thedate, season, values)
    open(filename, "a") do file
        print(file, "\"$(thedate)\",$(season)")
        for v in values
            print(file, ",$(round(v, 2))")
        end
        print(file, "\n")
    end
end

function save_futures_distribution(x)
    open(docpath("json/future_returns.json"), "w") do file
        write(file, "returns = ")
        JSON.print(file, map(i->round(i, 3), x))
        write(file, ";\n")
    end
end

In [ ]:
# The current date
thedate = Dates.today()

# Estimate for exchange rate
exchange_rate_distribution = ExchangeRate(Uniform(0.3, 0.5), Uniform(0.7, 0.72), 0.7, 0.7069, 0.68)

auction_plus_forecast_quantiles = calculate_forecasts(
                                                    thedate,
                                                    exchange_rate_distribution,
                                                    20000,
                                                    [0.1, 0.25, 0.5, 0.75, 0.9]
                                                )

In [ ]:
quantile(rand(exchange_rate_distribution, 1000), [0.1, 0.25, 0.5, 0.75, 0.9])

In [ ]:
# function getauctionplusquantiles(thedate)
#     # Estimate for exchange rate
#     if thedate < Dates.Date(2016,9,30)
#         exchange_rate_distribution = Uniform(0.65, 0.73)
#     else
#         exchange_rate_distribution = ExchangeRate(Uniform(0.4, 0.6), Uniform(0.69, 0.71), 0.7, 0.7069, 0.68)
#     end
#     calculate_forecasts(
#         thedate,
#         exchange_rate_distribution,
#         20000,
#         [0.1, 0.25, 0.5, 0.75, 0.9],
#         false
#     )
# end

# srand(11111)
# auctionplus = getauctionplusquantiles(Dates.Date(2016,6,2))' # hack first obs.
# srand(11111)
# for thedate in Dates.Date(2016,6,2):Dates.today()
#     Dates.day(thedate) == 1 && @show thedate
#     auctionplus = vcat(auctionplus, getauctionplusquantiles(thedate)')
# end
# plotribbon!(x, l, u, c, lab) = plot!(x, (l+u)/2, ribbon=(u-l)/2, c=c, label=lab)

# x = Dates.Date(2016,6,1):Dates.today()
# theme(:none)

# plot(
#     title="Historic Forecasts of Auction-Plus Model",
#     xlabel="Date of Forecast",
#     ylabel="Milk Price (\$/kgMS)",
#     ylims=(2, 9)
# )

# plotribbon!(x, auctionplus[:,1], auctionplus[:,5], colorant"lightgrey", "80\% Band (/10 - /90)")
# plotribbon!(x, auctionplus[:,2], auctionplus[:,4], colorant"darkgrey", "50\% Band (/25 - /75)")
# plot!(x, auctionplus[:,3], label="Median Guess (/50)", c=colorant"black")
# # savefig("auction_plus.pdf")
# open(datapath("model_parameters/auction_plus_forecasts.csv"), "w") do file
#     println(file, "\"Date\",\"Season\",\"x10_Percentile\",\"x25_Percentile\",\"x50_Percentile\",\"x75_Percentile\",\"x90_Percentile\"")
# end
# for i in 1:length(x)
#     append_auction_forecast(datapath("model_parameters/auction_plus_forecasts.csv"), x[i], 2016, auctionplus[i,:])
# end
# create_json("auction_plus_forecasts", vcat(1,3:7))

In [ ]:
append_auction_forecast(datapath("model_parameters/auction_plus_forecasts.csv"), thedate, 2016,auction_plus_forecast_quantiles)
create_json("auction_plus_forecasts", vcat(1,3:7))
# if NEW_AUCTION_ONLY
#     append_auction_forecast(datapath("model_parameters/auction_only_forecasts.csv"), thedate, 2016,auction_only_forecast_quantiles)
#     create_json("auction_only_forecasts", vcat(1,3:7))
# end

In [ ]:
auction_plus_forecast_quantiles_b = calculate_forecasts(
                                                    thedate,
                                                    exchange_rate_distribution,
                                                    20000,
                                                    linspace(0, 1, 5000),
                                                    false
                                                )
save_futures_distribution(rand(auction_plus_forecast_quantiles_b, 20000));

## Update the other forecasters

This section is just helper functions to update the website.

In [ ]:
function analyst_dictionary(filename, name, year=2016)
    df = @where(readtable(filename), :Season .== year)
    d = Dict{Symbol, Any}()
    d[:name] = name
    d[:x] = df[:Date]
    d[:y] = df[:Forecast]
    return d
end

function rebuild_other_analysts()
    open(docpath("json/other_analysts.json"), "w") do file
        write(file, "other_analysts = ")
        JSON.print(file,
            [
            analyst_dictionary(datapath("forecasts/agrihq.csv"), "AgriHQ"),
            analyst_dictionary(datapath("forecasts/asb.csv"), "ASB"),
            analyst_dictionary(datapath("forecasts/bnz.csv"), "BNZ"),
            analyst_dictionary(datapath("forecasts/fonterra.csv"), "Fonterra"),
            analyst_dictionary(datapath("forecasts/mkp.csv"), "NZX:MKPFU17"),
            analyst_dictionary(datapath("forecasts/westpac.csv"), "Westpac")
            ]
        )
        write(file, ";")
    end
    nothing
end
rebuild_other_analysts()

In [ ]:
function rebuild_future_params()
    open(docpath("json/future_params.json"), "w") do file
        latest_price = analyst_dictionary(datapath("forecasts/mkp.csv"), "NZX:MKPFU17")[:y][end]
        println(file, "hedge_quantity = 50;")
        println(file, "hedge_price    = $(latest_price);")
    end
    nothing
end
rebuild_future_params()

In [ ]:
function rebuild_spot()
    spot_price = get_spot_price()
    open(docpath("json/spot_price.json"), "w") do file
        write(file, "spot_price = ")
        JSON.print(file,
            Dict(
                :name => "Spot Price",
                :x => spot_price[:Date],
                :y => round(spot_price[:SpotPrice], 2)
            )
        )
        write(file, ";\n")
    end
    nothing
end
rebuild_spot()

In [ ]:
function rebuild_fonterra_data()
    df = getdata("forecasts/fonterra.csv")
    output = []
    by(df, :Season) do f
        push!(output, Dict{Symbol, Any}(:x=>f[:Date], :y=>f[:Forecast], :season=>f[:Season][1]))
        return 0
    end
    output
    open(docpath("json/fonterra_forecasts.json"), "w") do file
        write(file, "fonterra_data = ")
        JSON.print(file, output)
        write(file, ";")
    end
    nothing
end
rebuild_fonterra_data()

# GDT Forecaster

In [ ]:
nzx_months = ['F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X', 'Z']

# return the date of the n'th weekday in month-year
#   defaults to weekday=2 (Tuesday)
function getnthday(year, month, n, weekday=2)
    # loop through seven days
    for day in 1:7
        d = Dates.Date(year, month, day + (n-1)*7) 
        if Dates.dayofweek(d) == weekday
            return d
        end
    end
    # error to preserve type stability
    error("We couldn't find the right day in the week.")
end

# this errors for a few auctions 
# i.e. TE27 is actually a Wednesday...
# either errors in the data (from the Excel spreadsheet downloaded from GlobalDairyTrade
# or they shifted somes auctions if the 1st of the month was on a Wednesday...
# lets just ignore the differences for now
function getdateofauction(n)
    # we know TE27 happened on September 1st, 2010
    TE27 = Dates.Date(2010,9,1)
    if n < 27
        # before TE27, auctions happened on the first Tuesday only
        # calculate the approximate date (i.e. Year-Month) of the n'th TE
        te_nth = TE27 + Dates.Month(n - 27)
        # return the date
        return getnthday(Dates.year(te_nth), Dates.month(te_nth), 1, 2)
    else
        # odd numbered TE's happen on the first tuesday
        # even numbered TE's happen on the third tuesday
        nth = isodd(n)?1:3
        # calculate the approximate date (i.e. Year-Month) of the n'th TE
        #  we step forward 1 month every two auctions. But TE27 was the first
        #  auction in a month so we round down
        te_nth = TE27 + Dates.Month(floor(Int, (n - 27)/2))
        # return the date
        return getnthday(Dates.year(te_nth), Dates.month(te_nth), nth, 2)
    end
end

immutable EmpiricalError{T}
    x::Vector{T}
end
Base.rand(er::EmpiricalError) = rand(er.x)
immutable AR{N, T, T1, D}
    c::Tuple{Vararg{T, T1}}
    err::D
end
AR{T}(N::Int, c::Vector{T}, err) = AR{N, T, length(c), typeof(err)}(tuple(c...), err)

function step{N,T, M}(m::AR{N, T, M}, x0::AbstractVector{T})
    @assert length(x0) == N
    y = m.c[1] + rand(m.err)
    @inbounds for i in 1:N
        y += m.c[i+1] * x0[i]
    end
    y 
end
function simulatelog!{N,T, M}(y, m::AR{N, T, M}, x0::AbstractVector{T}, n)
    @assert length(x0) == N
    x = copy(x0)
    newval = zero(T)
    @inbounds for i in 1:n
        newval = step(m, x)
        for j in 2:N
            x[j] = x[j-1]
        end
        x[1] = newval
        y[i] = exp(newval)
    end
    y
end
function simulatelog{N,T, M}(m::AR{N, T, M}, x0::AbstractVector{T}, n)
    y = zeros(T, n)
    simulatelog!(y, m, x0, n)
    y
end
function lag(df, lag, id, fixedcols)
    new_df = copy(df)
    for i in 1:lag
        dfi = copy(df)
        dfi[id] += i
        new_df = join(new_df, dfi, on=[id])
        for col in fixedcols
            delete!(new_df, Symbol("$(col)_1"))
        end
    end
    new_df
end

In [ ]:
gdt = @select(getdata("gdt/events.csv"), :TE, :SMP, :WMP)
sort!(gdt, cols=[:TE])
gdt[:id] = 1:size(gdt, 1)
gdt[:WMP] = log.(gdt[:WMP])
gdt[:SMP] = log.(gdt[:SMP])
lagged_prices = lag(gdt, 2, :id, [:TE])

# wmp_model = fit(LinearModel, @formula(WMP ~ WMP_1 + WMP_2), lagged_prices)
wmp_model = fit(LinearModel, WMP ~ WMP_1 + WMP_2, lagged_prices)
wmp_ar = AR(2, coef(wmp_model), residuals(wmp_model))
# smp_model = fit(LinearModel, @formula(SMP ~ SMP_1 + SMP_2), lagged_prices)
smp_model = fit(LinearModel, SMP ~ SMP_1 + SMP_2, lagged_prices)
smp_ar = AR(2, coef(smp_model), residuals(smp_model))

function adddata!(data, model, x0, lastauction)
    n = 20_000
    Y = Array(Float64, (24, n))
    for i in 1:n
        simulatelog!(view(Y,:, i), model, x0, 24)
    end
    Z = Array(Float64, (size(Y, 1), 3))
    for i in 1:size(Y, 1)
        Z[i, :] = quantile(Y[i, :], [0.1, 0.5, 0.9])
    end
    data["date"]   = [getdateofauction(lastauction + i) for i in 1:size(Y, 1)]
    data["lower"]  = round.(Z[:, 1], 2)
    data["middle"] = round.(Z[:, 2], 2)
    data["upper"]  = round.(Z[:, 3], 2)
    
end
lastauction = maximum(gdt[:TE])
data = Dict()
models = Dict("WMP" => wmp_ar, "SMP" => smp_ar)
for prod in ["WMP", "SMP"]
    data[prod] = Dict()
    adddata!(data[prod], models[prod], reverse(collect(gdt[(end-1):end, Symbol(prod)])), lastauction)
end
open(docpath("json/future_price_forecast.json"), "w") do file
    println(file, "future_price_forecast = ")
    JSON.print(file, data)
    println(file, ";")
end

In [ ]:
# df = @select(getdata("nzx/dsp_complete.csv", :Date, "dd/mm/yyyy"), :Contract, :Daily_Settlement_Price, :Date)
# rename!(df, Dict(:Date=>:Trade_date, :Contract => :Code, :Daily_Settlement_Price=>:Calculated_DSP))

# new_df = @select(getdata("nzx/WMPfuture.csv", :Trade_date), :Code, :Calculated_DSP, :Trade_date);
# append!(new_df, @select(getdata("nzx/SMPfuture.csv", :Trade_date), :Code, :Calculated_DSP, :Trade_date))

# append!(df, new_df)

# df=df[isna.(df[:Calculated_DSP]) .< 0.5, :]

# df[:Product] = map(s->s[1:4], df[:Code])
# df = df[(df[:Product] .== "WMPF") | (df[:Product] .== "SMPF"), :]
# df[:Year] = map(s->parse(Int, s[6:end])+2000, df[:Code])
# df[:Month] = map(s->findin(nzx_months, s[5])[1], df[:Code])

# # get third wednesday
# df[:SettlementDate] = map((y,m)->getnthday(y, m, 3, 3), df[:Year], df[:Month])
# df = @where(df, :Trade_date .<= :SettlementDate)

# completed_df = df[(df[:Year] .< 2017) | (df[:Month] .< Dates.month(Dates.today())), :]

# finalprice = by(completed_df, :Code) do tmp
#     lasttradedate = maximum(tmp[:Trade_date])
#     DataFrame(
#     FinalDate = lasttradedate,
#     FinalPrice=@where(tmp, :Trade_date .== lasttradedate)[:Calculated_DSP][1]
#     )
# end
# completed_df = join(completed_df, finalprice, on=:Code)
# completed_df = @select(completed_df, :Product, :Year, :Month, :Code, :Trade_date, :Calculated_DSP, :FinalDate, :FinalPrice)
# sort!(completed_df, cols=[:Product, :Code, :Trade_date])
# completed_df[:DateDistance] = map((d1, d2) -> round(Int, (d1 - d2).value), completed_df[:FinalDate], completed_df[:Trade_date])
# completed_df[:Error] = completed_df[:FinalPrice] - completed_df[:Calculated_DSP]
# completed_df[:WeekDistance] = round(Int, completed_df[:DateDistance] / 7)
# unique!(completed_df)
# completed_df[1,:]

In [ ]:
# plot(by(@where(completed_df, :WeekDistance .<= 30), [:FinalDate, :Product, :Code]) do _
#     sum((_[:FinalPrice]./_[:Calculated_DSP]-1).^2) / size(_, 1)
#     end,
# :FinalDate, :x1, group=:Product)

In [ ]:
# historical_accuracy = by(completed_df, [:Product, :WeekDistance]) do tmp
#     DataFrame(
#     lower = quantile(tmp[:Error], 0.1),
#     middle = quantile(tmp[:Error], 0.5),
#     upper = quantile(tmp[:Error], 0.9)
#     )
# end